In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
import functools


In [21]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}


In [22]:
training = pd.read_csv("/Users/wxu/workspace/regression/week5/kc_house_train_data.csv",sep=",",header=0,dtype = dtype_dict)
testing = pd.read_csv("/Users/wxu/workspace/regression/week5/kc_house_test_data.csv",sep=",",header=0,dtype = dtype_dict)
house = pd.read_csv("/Users/wxu/workspace/regression/week5/kc_house_data.csv",sep=",",header= 0,dtype = dtype_dict)


In [4]:
def get_numpy_data(dataframe, features, output):
    
    dataframe['constant']=1
    features = ['constant'] + features
    features_matrix = dataframe[features].as_matrix()
    output_array = dataframe.as_matrix([output])
    
    return (features_matrix, output_array)

In [5]:
def predict_outcome(feature_matrix,weights):
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [6]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    norm_features = feature_matrix/norms
    return (norm_features,norms)

In [7]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # i: the ith feature
    # feature_matrix/output, output from get_numpy_data and normalize_features
    #weights: initial weight
    # compute prediction
    prediction = predict_outcome(feature_matrix,weights)
    prediction.shape = (len(prediction),1)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    curr_feature = np.asarray(map(lambda x: x[i],feature_matrix))
    curr_feature.shape = (len(curr_feature),1)
    error_no_i = output - prediction + weights[i]*curr_feature
    ro_i = sum(curr_feature*error_no_i)[0]
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i+l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i-l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i


In [8]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    num_of_feature = feature_matrix.shape[1]
    curr_weights = initial_weights
    change = [0]*num_of_feature
    for i in range(num_of_feature):
        tmp = curr_weights[i]
        curr_weights[i] = lasso_coordinate_descent_step(i, feature_matrix = feature_matrix , 
                                                        output=output, weights=curr_weights, 
                                              l1_penalty=l1_penalty)
        
        change[i]=abs(curr_weights[i]-tmp)         
        
    while max(change)>tolerance:
        change = [0]*num_of_feature
        for i in range(num_of_feature):
            tmp = curr_weights[i]
            curr_weights[i] = lasso_coordinate_descent_step(i, feature_matrix = feature_matrix , 
                                                        output=output, weights=curr_weights,l1_penalty=l1_penalty)
            change[i]=abs(curr_weights[i]-tmp)
           
    return curr_weights

In [9]:
twoFeatures = ["sqft_living","bedrooms"]
moreFeatures = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view", 
                "condition", "grade", "sqft_above", "sqft_basement", "yr_built", "yr_renovated"]


In [29]:
train_twoFeatures,train_output = get_numpy_data(training,twoFeatures,"price")
train_twoFeatures_norm,norms_twoFeatures = normalize_features(train_twoFeatures)

train_moreFeatures,train_output = get_numpy_data(training,moreFeatures,"price")
train_moreFeatures_norm,norms_moreFeatures = normalize_features(train_moreFeatures)

test_twoFeatures,test_output = get_numpy_data(testing,twoFeatures,"price")
test_moreFeatures,test_output = get_numpy_data(testing,moreFeatures,"price")


In [26]:
weights1e7 = lasso_cyclical_coordinate_descent(feature_matrix = train_moreFeatures_norm, 
                                               output = train_output, initial_weights = [0]*14,
                                               l1_penalty=1e7, tolerance=1.0)
weights1e8 = lasso_cyclical_coordinate_descent(feature_matrix = train_moreFeatures_norm, 
                                               output = train_output, initial_weights = [0]*14,
                                               l1_penalty=1e8, tolerance=1.0)
weights1e4 = lasso_cyclical_coordinate_descent(feature_matrix = train_moreFeatures_norm, 
                                               output = train_output, initial_weights = [0]*14,
                                               l1_penalty=1e4, tolerance=5e5)

print weights1e7
print weights1e8
print weights1e4

[24429600.234403368, 0.0, 0.0, 48389174.771548547, 0.0, 0.0, 3317511.214921644, 7329961.8117143307, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[71114625.714887127, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[78564738.341568649, -22097398.924305156, 12791071.872784993, 93808088.092812315, -2013172.7570497482, -4219184.9326500809, 6482842.8175350353, 7127408.5348068392, 5001664.8546970822, 14327518.437141119, -15770959.152374173, -5159591.2221315242, -84495341.768439025, 2824439.4970368915]


In [28]:
weights1e7_normalize = weights1e7/norms_moreFeatures
weights1e8_normalize = weights1e8/norms_moreFeatures
weights1e4_normalize = weights1e4/norms_moreFeatures

In [37]:
predict1e7 = predict_outcome(test_moreFeatures,weights1e7_normalize);predict1e7.shape = [4229,1]
predict1e8 = predict_outcome(test_moreFeatures,weights1e8_normalize);predict1e8.shape = [4229,1] 
predict1e4 = predict_outcome(test_moreFeatures,weights1e4_normalize);predict1e4.shape = [4229,1]
rss1e7 = sum((predict1e7-test_output)**2)
rss1e8 = sum((predict1e8-test_output)**2)
rss1e4 = sum((predict1e4-test_output)**2)
print rss1e7;print rss1e8; print rss1e4

[  2.75962076e+14]
[  5.37166151e+14]
[  2.28459959e+14]


In [50]:
#quiz 1 quiz2
features_whole,output_whole = get_numpy_data(house,twoFeatures,"price")
features_whole_normalize,whole_norms = normalize_features(features_whole)
initial_weights = [1,4,1]
predict_whole = predict_outcome(features_whole_normalize,initial_weights)
feature1 = features_whole_normalize[:,1];feature2 = features_whole_normalize[:,2]
predict_whole.shape = [21613,1];feature1.shape = [21613,1];feature2.shape = [21613,1]
#error_no_i = output - prediction + weights[i]*curr_feature
ro1 = sum(feature1*(output_whole - predict_whole + initial_weights[1]*feature1))
ro2 = sum(feature2*(output_whole - predict_whole + initial_weights[2]*feature2))
print ro1 
print ro2

[ 87939470.82325152]
[ 80966698.66623905]


In [51]:
#quiz3,4
simple_weights = lasso_cyclical_coordinate_descent(feature_matrix = features_whole_normalize, 
                                               output = output_whole, initial_weights = [0]*3,
                                               l1_penalty=1e7, tolerance=1.0)

predict_simple_whole = predict_outcome(features_whole_normalize,simple_weights);predict_simple_whole.shape = [21613,1]
rss_simple = sum((predict_simple_whole-output_whole)**2)
print rss_simple
print simple_weights

[  1.63049248e+15]


[21624997.959518719, 63157247.20788978, 0.0]
